In [1]:
import numpy as np
import cv2
import scipy as scipy
import matplotlib.pyplot as plt
from astropy.io import fits
import polarTransform
from astropy.utils.data import get_pkg_data_filename
from scipy.stats import wasserstein_distance
from marvin.tools import Maps
from marvin.utils.general.images import showImage

[INFO]: No release version set. Setting default to DR17
[WARNING]: path /Users/runquanguan/sas/mangawork/manga/spectro/redux/v3_1_1/drpall-v3_1_1.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /Users/runquanguan/sas/mangawork/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits cannot be found. Setting dapall to None. (MarvinUserWarning)
[WARNING]: Passing field metadata as keyword arguments is deprecated. Use the explicit `metadata=...` argument instead. Additional metadata: {'location': 'view_args'} (RemovedInMarshmallow4Warning)
[WARNING]: The 'missing' argument to fields is deprecated. Use 'load_default' instead. (RemovedInMarshmallow4Warning)


In [5]:
# Find the nth occurance to resolve the repetition of '-'
# For later use
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [1]:
def generate_profile_histogram(PATH):
    
    # Input: File Path
    # Outout: Smoothened histogarm, plateifu
    
    # Load MAPS:
    image_file = get_pkg_data_filename(PATH)
    
    # Gaussian-fitted equivalent widths measurements 
    EMLINE_GEW = fits.getdata(image_file, ext=33)
    
    # Transform to polar coordinate
    cartesianImage = EMLINE_GEW[16]
    # Set center of image(just in case)
    w = EMLINE_GEW[16].shape[0]
    h = EMLINE_GEW[16].shape[1]
    # Generate polarImage
    polarImage, ptSettings = polarTransform.convertToPolarImage(EMLINE_GEW[16], center=[round(w/2), round(h/2)])
    
    # Integrate the column
    EW_COL = [sum(x)/polarImage.T.shape[0] for x in zip(*polarImage.T)]

    # Exclude outlier using 3-sigma variant
    mean = np.mean(EW_COL)
    sd = np.std(EW_COL)
    EW_CLEAN = [x for x in EW_COL if (x > mean - 2 * sd)]
    EW_CLEAN = [x for x in EW_CLEAN if (x < mean + 2 * sd)]
    # Smoothening the curve using Gaussian filter
    EW_SMO = scipy.ndimage.gaussian_filter(EW_CLEAN, sigma = 5)

    # To better identify the feature, plot two cycles of the galaxy
    EW_SMO_TW = list(np.append(EW_SMO, EW_SMO))

    # limited the array to exactly one cycle: but from min to min
    min_index = min(EW_SMO_TW) # find min value
    start_index = [i for i, n in enumerate(EW_SMO_TW) if n == min_index][0] # find index of 1st min
    end_index = [i for i, n in enumerate(EW_SMO_TW) if n == min_index][1] # find index of 2nd min
    trunc_EW = np.array(EW_SMO_TW[start_index:end_index]) # truncate from 1st to 2nd min
    
    # Normalization 
    norm = np.linalg.norm(trunc_EW)
    norm_EW = trunc_EW/norm
    
    # Make them all to the same length through interpolation
    x = np.linspace(0,len(norm_EW),len(norm_EW))
    y = norm_EW
    x2 = np.linspace(0,len(norm_EW),500)
    f_linear = scipy.interpolate.interp1d(x, y)
    intp_EW = f_linear(x2)
    
    #extract plateifu
    idx1 = find_nth(PATH, '-', 2) 
    idx3 = find_nth(PATH, '-', 4)
    plateifu = PATH[idx1+1:idx3]
    
    
    
    return intp_EW, plateifu

In [ ]:
def calculate_maximum(test):
    
    # Calculate the loss between the test galaxy and the 17 bicone galaxy
    
    # import test data
    test_EW_hist = test[0]
    test_plateifu = test[1]
    
    # Load examples
    BC_PATH = '/Users/runquanguan/Documents/Research/MaNGA-AGN/Pipeline&Instrction/obvious_bicone_feature_position.fits'
    FOLDER = '/Users/runquanguan/Documents/Research/MaNGA-AGN/Data/'
    hdul = fits.open(BC_PATH)
    hdu = hdul[1].data
    
    total_loss = []
    for data in hdu:
        plateifu = str(data[0])
        FILENAME = 'manga-' + plateifu + '-MAPS-SPX-MILESHC-MASTARSSP.fits'
        FILE_PATH = FOLDER+FILENAME
        # Define arrays
        BC_EW_hist = generate_profile_histogram(FILE_PATH)[0]
        # Find the loss using Earth Moving Distance
        loss = wasserstein_distance(test_EW_hist, BC_EW_hist)
        total_loss.append(loss)
        
    test_plateifu = test[1]
    
    # Check if test galaxy exist in bicone list
    
    if test_plateifu in hdu['PLATEIFU']:
        final_loss = sum(total_loss)/(hdu.shape[0]-1)
    else:
        final_loss = sum(total_loss)/hdu.shape[0]
        
    return final_loss
    
    
    
    
    